In [1]:
import random
import copy
from math import sqrt

from tqdm import tqdm
import pandas as pd

In [2]:
class kd_node():
    def __init__(self, x: float, y: float, dim=0, parent=None):
        self.x = x
        self.y = y
        self.coords = [x, y]
        self.dim = dim
        self.parent = parent

        self.branches = {
            'up': None,
            'low': None
        }

    def in_low_branch(self, x, y):
        p = [x,y]
        return p[self.dim] < self.coords[self.dim]

# =============================================================================

    def add_element(self, x, y):
        flag = 'low' if self.in_low_branch(x, y) else 'up'

        if self.branches[flag] is None :
            dim = (self.dim + 1) % 2
            self.branches[flag] = kd_node(x, y, dim, parent = self)
        else:
            self.branches[flag].add_element(x, y)

# =============================================================================

    def delete_element(self, x, y):
        if (self.x == x) and (self.y == y):
            if self.branches['up'] is not None:
                min_point = self.branches['up'].find_min(self.dim)
                self.replace_self(min_point)

                self.branches['up'].delete_element(self.x, self.y)

            elif self.branches['low'] is not None:
                min_point = self.branches['low'].find_min(self.dim)
                self.replace_self(min_point)

                self.branches['up'] = self.branches['low']
                self.branches['low'] = None
                self.branches['up'].delete_element(self.x, self.y)

            else:
                self.delete_self()
            return True

        elif self.in_low_branch(x,y):
            if self.branches['low'] is not None:
                return self.branches['low'].delete_element(x,y)
            else:
                return False
        else:
            if self.branches['up'] is not None:
                return self.branches['up'].delete_element(x,y)
            else:
                return False

    # -----------------------------------------------

    def replace_self(self, point):
        self.x, self.y = point.x, point.y
        self.coords = [self.x, self.y]

    def delete_self(self):
        self.parent.cut_leaf(self)

    def cut_leaf(self, leaf):
        for x in self.branches:
            if self.branches[x] is leaf:
                self.branches[x] = None
                return

    # -----------------------------------------------

    def find_min(self, dim):
        if self.dim == dim:
            if self.branches['low'] is None:
                return self
            else:
                return self.branches['low'].find_min(dim)
        else:
            point = self
            if self.branches['low'] is not None:
                min_left = self.branches['low'].find_min(dim)
                point = self.min_on_dimension(point, min_left, dim)
            if self.branches['up'] is not None:
                min_right = self.branches['up'].find_min(dim)
                point = self.min_on_dimension(point, min_right, dim)
            return point

    def min_on_dimension(self, point_a, point_b, dim):
        if point_a.coords[dim] <= point_b.coords[dim]:
            return point_a
        else:
            return point_b

# =============================================================================

    def search_element(self, x, y):
        if (self.x == x) and (self.y == y):
            return True
        elif self.in_low_branch(x,y):
            if self.branches['low'] is not None:
                return self.branches['low'].search_element(x,y)
            else:
                return False
        else:
            if self.branches['up'] is not None:
                return self.branches['up'].search_element(x,y)
            else:
                return False

# =============================================================================

    def knn_search(self, x, y, k=1, bounding_box=None, current_best=None):
        if bounding_box is None:
            bounding_box = dict()
            for i in range(len(self.coords)):
                bounding_box[i] = {'up': None,
                'low': None}

        flag = self.in_low_branch(x,y)
        flag_a = 'low' if flag else 'up'
        flag_b = 'up' if flag else 'low'

        if current_best is None:
            if self.branches[flag_a] is not None:
                bb = self.calc_branch_bounding_box(bounding_box, flag_a)
                current_best = self.branches[flag_a].knn_search(x, y, k, bb)
                current_best = self.check_self_dist(x, y, k, current_best)
            else:
                dist = sqrt((x - self.x)**2 + (y - self.y)**2)
                current_best = [{
                    'point': self,
                    'distance': dist
                }]

        else:
            current_best = self.check_self_dist(x, y, k, current_best)

            if self.branches[flag_a] is not None:
                bb = self.calc_branch_bounding_box(bounding_box, flag_a)
                if self.check_branch(x, y, current_best=current_best, bbox=bb):
                #check branch -- pass cur best
                    current_best = self.branches[flag_a].knn_search(x, y, k, bb, current_best=current_best)
            
        if self.branches[flag_b] is not None:
            bb = self.calc_branch_bounding_box(bounding_box, flag_b)
            if self.check_branch(x, y, current_best=current_best, bbox=bb):
                #check branch -- pass cur best
                current_best = self.branches[flag_b].knn_search(x, y, k, bb, current_best=current_best)
                    
        return current_best

    # -----------------------------------------------

    def calc_branch_bounding_box(self, boundind_box, branch):
        bb = copy.deepcopy(boundind_box)
        bb[self.dim][branch] = self.coords[self.dim]
        return bb

    def check_self_dist(self, x, y, k, current_best):
        dist = sqrt((x - self.x)**2 + (y - self.y)**2)
        if len(current_best) < k:
            point = {
                'point': self,
                'distance': dist
            }
            current_best.append(point)
            return current_best
        else:
            worst_best_d = -1
            worst_id = None
            for i in range(len(current_best)):
                if current_best[i]['distance'] > worst_best_d:
                    worst_best_d = current_best[i]['distance']
                    worst_id = i
            
            if worst_best_d > dist:
                point = {
                    'point': self,
                    'distance': dist
                }
                current_best.append(point)
                del current_best[worst_id]
            return current_best

    def check_branch(self, x, y, current_best, bbox):
        worst_best_d = -1
        for i in range(len(current_best)):
            if current_best[i]['distance'] > worst_best_d:
                worst_best_d = current_best[i]['distance']

        coords = [x, y]
        dist = 0
        for dim in bbox:
            if (bbox[dim]['up'] is not None)  and (bbox[dim]['up'] > coords[dim]):
                dim_dist = (bbox[dim]['up'] - coords[dim]) ** 2
                dist += dim_dist
            elif (bbox[dim]['low'] is not None)  and (bbox[dim]['low'] < coords[dim]):
                dim_dist = (bbox[dim]['low'] - coords[dim]) ** 2
                dist += dim_dist
        dist = sqrt(dist)
        return (dist < worst_best_d)

# =============================================================================

    def storage(self, prev_list=None):
        if prev_list is None:
            prev_list = []

        point = (self.x, self.y)
        prev_list.append(point)

        for i in self.branches:
            if self.branches[i] is not None:
                prev_list = self.branches[i].storage(prev_list)

        return prev_list

# =============================================================================

    def build(self, point_list):
        if len(point_list) >= 5:
            cols = point_list.columns.tolist()
            df_dict = {
                'up': point_list[point_list[cols[self.dim]] >= self.coords[self.dim]],
                'low': point_list[point_list[cols[self.dim]] < self.coords[self.dim]]
            }

            for key in self.branches:
                if len(df_dict[key]):
                    if self.branches[key] is None:
                        check_dim = (self.dim + 1) % len(self.coords)
                        df_dict[key]['calc'] = (df_dict[key][cols[check_dim]] - df_dict[key][cols[check_dim]].median()).abs()

                        ind = df_dict[key][df_dict[key]['calc'] == df_dict[key]['calc'].min()].index[0]
                        point = df_dict[key].loc[ind, :]
                        self.add_element(point['x'], point['y'])
                        df_dict[key].drop(ind, inplace=True)

                    self.branches[key].build(df_dict[key])
            del df_dict

        else:
            for key in point_list.index:
                point = point_list.loc[key, :]
                self.add_element(point['x'], point['y'])
        del point_list

# =============================================================================

    def print(self, lv=0):
        print(f"[{self.x}, {self.y}]", end='')
        flag = False
        for x in self.branches:
            if self.branches[x] is not None:
                flag = True
                break
        if not flag:
            print('')
            return
        else:
            print(':')
            for x in self.branches:
                blank = ''
                for i in range(lv+1):
                    blank += '  '
                print(f"{blank}{x}:", end='')
                
                if self.branches[x] is None:
                    print("-null-")
                else:
                    self.branches[x].print(lv+1)

In [3]:
class kd_tree():
    def __init__(self):
        self.root = None
    
    def add_element(self, x, y):
        if self.root is not None:
            self.root.add_element(x, y)
        else:
            self.root = kd_node(x, y, dim=0, parent=self)
    
    def delete_element(self, x, y):
        if self.root is not None:
            return self.root.delete_element(x,y)
        else:
            return False
    
    def cut_leaf(self, leaf):
        self.root = None   
    
    def storage(self):
        if self.root is not None:
            return self.root.storage()
        else:
            return False

    def build(self, point_list):        
        point_list = pd.DataFrame(point_list, columns=['x', 'y'])

        if self.root is None:
            point_list['calc'] = (point_list['x'] - point_list['x'].median()).abs() + (point_list['y'] - point_list['y'].median()).abs()
            ind = point_list[point_list['calc'] == point_list['calc'].min()].index[0]
            point = point_list.loc[ind, :]

            self.root = kd_node(point['x'], point['y'], dim=0, parent=self)
            point_list.drop(ind, inplace=True)
        
        self.root.build(point_list)

    def update(self):
        pass

    def search(self):
        if self.root is not None:
            return self.root.search_element(x,y)
        else:
            return False

    def knn_search(self, x, y, k=1):
        if self.root is not None:
            return self.root.knn_search(x,y,k)
        else:
            return []

In [5]:
k2 = kd_tree()
points = [(random.randrange(200), random.randrange(200)) for _ in range(1000)]
k2.build(points)

 calc
688  190  20   0.5
730  199  21   0.5
996  193  20   0.5
7
ey
      x   y  calc
85  192  35   0.0
4
oh
       x   y  calc
466  194  17   0.0
666  194  17   0.0
1
oh
       x   y  calc
360  183  30   1.0
917  172  28   1.0
7
ey
       x   y  calc
236  178  37   0.0
2
oh
       x   y  calc
154  178  18   0.5
534  179  18   0.5
3
oh
       x   y  calc
783  185  10   0.0
12
ey
       x   y  calc
275  199  13   4.5
292  199   4   4.5
298  193   4   4.5
801  186  13   4.5
5
ey
       x   y  calc
131  192  14   3.0
801  186  13   3.0
1
oh
       x  y  calc
184  195  1   0.0
2
oh
       x   y  calc
271  172  13   4.0
567  175  13   4.0
859  172   5   4.0
5
ey
       x   y  calc
567  175  13   0.5
792  174  14   0.5
1
oh
       x  y  calc
715  178  4   0.0
2
oh
       x   y  calc
399  164  17   0.5
754  161  18   0.5
27
ey
       x   y  calc
133  157  32   2.5
530  157  22   2.5
563  152  34   2.5
611  152  22   2.5
635  152  29   2.5
923  152  35   2.5
13
ey
       x   y  calc
372  160  

In [ ]:
kdtree = kd_node(random.random(),random.random())

In [ ]:
for i in range(10000):
    x = random.randrange(500)
    y = random.randrange(500)
    print(f"({x}, {y})", end='')
    kdtree.add_element(x, y)
print('\ndone')


In [ ]:
kdtree.print()

In [ ]:
if kdtree.delete_element(77, 56):
    print("DUCK yes")
else:
    print("dammit")

In [ ]:
for i in kdtree.knn_search(2,53.5, k=30):
    print(f"[{i['point'].x}, {i['point'].y}] - Dist. {i['distance']}")